In [36]:
using Plots
using LinearAlgebra
using Statistics
using DifferentialEquations

In [ ]:
#physical constants for rate expressions
const a1 = 10^4.2
const ea1 = 4.01e4
const a2 = 10^13.23
const ea2 = 1.2804e5
const a3 = 10^6.78
const ea3 = 1.12e5
const R = 8.314
const gc = 1.0
const ϕ = 0.5
const μ = 0.025
const ρc = 3950

In [60]:
function arr(ao, ea, T) 
    ao*exp(-ea/8.314/T) 
end

function k4f(T) 
    1000*exp(17.13-13000/(1.987*T))
end

function k4b(T) 
    exp(5.4+16000/(1.987*T))
end

k4b (generic function with 1 method)

We begin with the equations for the rates:
$$\frac{\partial O_2}{\partial t} = -\frac{1}{2}r_1 - \frac{1}{2}r_2 - 3r_3 - r_{4f} + r_{4r}$$
$$\frac{\partial Et}{\partial t} = -r_1-r_3$$
$$\frac{\partial HCl}{\partial t} = -2r_1 - r_2 - 4r_{4f} + 4r_{4r}$$
$$\frac{\partial EtCl_2}{\partial t} = r_1 - r_2$$
$$\frac{\partial H_2O}{\partial t} = r_1 + r_2 + 2r_3 + 2r_{4f} - 2r_{4r}$$
$$\frac{\partial EtCl_3}{\partial t} = r_2$$
$$\frac{\partial CO_2}{\partial t} = 2r_3$$
$$\frac{\partial Cl_2}{\partial t} = 2r_{4f} - 2r_{4r}$$

Then we get the rate equations from Lakshmanan:
$$r_1 = k_1p_{Et}p_{Cl_2}^{0.5}$$
$$r_2 = k_2p_{EtCl_2}p_{Cl_2}^{0.5}$$
$$r_3 = k_3p_{0_2}p_{Cl_2}^{0.5}p_{Et}$$
$$r_4 = K_4p_{O_2}p_{Cl_2}^{-1} \; K_4 = \frac{1000*exp(17.13-13000/(1.987*T))}{exp(5.4+16000/(1.987*T))}$$
Using the work of Falconer and Brown we can restate the 8 equations above in the terms: $\frac{\partial F_i}{\partial V}$



$$\frac{\partial F_{O_2}}{\partial V} = -\frac{1}{2} k_1 (\frac{P}{F_{tot}RT})^{3/2}F_{Et}F_{Cl_2}^{0.5} - \frac{1}{2} k_2 (\frac{P}{F_{tot}RT})^{3/2}F_{Et}F_{Cl_2}^{0.5} - 3k_3 (\frac{P}{F_{tot}RT})^{2.5} F_{O_2}F_{Cl_2}^{0.5}F_{Et} - \frac{1}{2}k_{4f}F_{O_2}F_{Cl_2}^{-1} + \frac{1}{2}k_{4b}F_{O_2}F_{Cl_2}^{-1}$$
$$\frac{\partial F_{Et}}{\partial V} = -k_1 (\frac{P}{F_{tot}RT})^{3/2}F_{Et}F_{Cl_2}^{0.5} - k_3 (\frac{P}{F_{tot}RT})^{2.5} F_{O_2}F_{Cl_2}^{0.5}F_{Et}$$
$$\frac{\partial F_{HCl}}{\partial V} = -2k_1 (\frac{P}{F_{tot}RT})^{3/2}F_{Et}F_{Cl_2}^{0.5} - k_2 (\frac{P}{F_{tot}RT})^{3/2}F_{Et}F_{Cl_2}^{0.5} - 2k_{4f}F_{O_2}F_{Cl_2}^{-1} + 2k_{4b}F_{O_2}F_{Cl_2}^{-1}$$
$$\frac{\partial F_{EtCl_2}}{\partial V} = k_1(\frac{P}{F_{tot}RT})^{3/2}F_{Et}F_{Cl_2}^{0.5} - k_2 (\frac{P}{F_{tot}RT})^{3/2}F_{Et}F_{Cl_2}^{0.5}$$
$$\frac{\partial F_{H_2O}}{\partial V} = k_1 (\frac{P}{F_{tot}RT})^{3/2}F_{Et}F_{Cl_2}^{0.5} + k_2 (\frac{P}{F_{tot}RT})^{3/2}F_{Et}F_{Cl_2}^{0.5} + 2k_3 (\frac{P}{F_{tot}RT})^{2.5} F_{O_2}F_{Cl_2}^{0.5}F_{Et} + 2k_{4f}F_{O_2}F_{Cl_2}^{-1} - 2k_{4b}F_{O_2}F_{Cl_2}^{-1}$$
$$\frac{\partial F_{EtCl_3}}{\partial V} = k_2 (\frac{P}{F_{tot}RT})^{3/2}F_{Et}F_{Cl_2}^{0.5}$$
$$\frac{\partial F_{CO_2}}{\partial V} = 2k_3 (\frac{P}{F_{tot}RT})^{2.5} F_{O_2}F_{Cl_2}^{0.5}F_{Et}$$
$$\frac{\partial F_{Cl_2}}{\partial V} = k_{4f}F_{O_2}F_{Cl_2}^{-1} + k_{4b}F_{O_2}F_{Cl_2}^{-1}$$

Now we can use Falconer and Brown's example of how to solve for $\frac{dT}{dV}$ for the working fluid and then use $m_{in} c_{p, in} \Delta T_{in} = m_{out}c_{p,out}\Delta T_{out}$ to solve for the temperature of the coolant

Finally we use the Ergun equation to solve for:
$$\frac{\partial P}{\partial V} = \frac{\partial P}{\partial W} \cdot \frac{\partial W}{\partial V}$$
$$\frac{\partial W}{\partial V} = \rho_{cat}(1-\phi)$$
$$\frac{\partial P}{\partial W} = \frac{-\alpha}{2} \frac{T}{T_0} \frac{P_0}{P/P_0} \frac{F_t}{F_t0}$$

In [80]:
#our order of u will be 8 materials kgmol/hr, working fluid temp K, coolant temp K, pressure kPa 
#parameters will be vector of cps in kJ/kgmol-K, vector of DeltaH's in kJ/kgmol, tube diameter, number tubes,
#Mw vector, Po, To, Fto
function vinylCh(du,u,p,t) 
    a1 = 10^4.2
    ea1 = 4.01e4
    a2 = 10^13.23
    ea2 = 1.2804e5
    a3 = 10^6.78
    ea3 = 1.12e5
    R = 8.314
    gc = 1.0
    ϕ = 0.5
    μ = 0.025
    ρc = 3950
    
    fTot = sum(u[1:8])
    α = u[11]/(fTot*R*u[9]) #using to avoid recomputing this 
    β = u[2]*sqrt(u[8]) #This gets used for reactions 1 and 2
    γ = u[1]*sqrt(u[8])*u[2] #reaction 3
    δ = u[1]/u[8] #reaction 4
    k1 = arr(a1, ea1, u[9])
    k2 = arr(a2, ea2, u[9])
    k3 = arr(a3, ea3, u[9])
    k4for = k4f(u[9])
    k4bac = k4b(u[9])
    fCpTot = sum(u[1:8] .* p[1])
    hRxn = k1*u[2]*-p[2][1]+k2*u[4]*-p[2][2]+k3*u[2]*-p[2][3]+k4for*u[1]*-p[2][4]+k4bac*u[1]*p[2][5]
    Ac = (p[3]/4)^2*pi*p[4]
    G = sum(u[1:8] ./ p[5])/Ac
    bet = G*(1-ϕ)/(α*mean(p[5])*1000*gc*1/8*p[3]*ϕ^3) * ((150*(1-ϕ)*μ)/(1/8*p[3])+1.75*G)
    alp = 2*bet/(Ac*ρc*(1-ϕ)*p[6])
    
    
    du[1] = -0.5*k1*α^1.5*β - -0.5*k2*α^1.5*β - 3*k3*α^2.5*γ - 0.5*k4for*γ + 0.5*k4bac*γ
    du[2] = -k1*α^1.5*β - k3*α^2.5*γ
    du[3] = -2*k1*α^1.5*β - k2*α^1.5*β - 2*k4for*δ + 2*k4bac*δ
    du[4] = k1*α^1.5*β - k2*α^1.5*β
    du[5] = k1*α^1.5*β + k2*α^1.5*β +2*k3*α^2.5*γ* + 2*k4for*δ - 2*k4bac*δ
    du[6] = k2*α^1.5*β
    du[7] = 2*α^2.5*γ
    du[8] = k4for*δ - k4bac*δ
    du[9] = hRxn/fCpTot - 4*300*(u[9]-u[10])/p[3]
    du[10] = -fCpTot*du[9]/(875.88*100) #using 100kgmol/hr of dowtherm A 
    du[11] = -alp/2 * u[9]/p[7] * p[6]/(u[11]/p[6]) * sum(u[1:8])/p[8] * ρc*(1-ϕ)
end

vinylCh (generic function with 1 method)

In [81]:
u0 = [970.24,970.24,2910.75,0.0,0.0,0.0,0.0,5.82,500,400,2000e3]
diam = 0.02
tubes = 100
vspan = (0.0,3.6*sum(u0[1:8])/100)
cps = [0.0293,0.0427,0.0291,0.0772,0.0336,0.0846,0.0371,0.0336] .* 1000
dH = [-236.22,-165.15,-1299.78,-111.97,111.97] .* 1000
Mw = [32,26.04,36.46,96.94,18,131.38,44.01,70.9]
po = 2000
to = 500
fto = sum(u0[1:8])
p = [cps,dH,diam,tubes,Mw,po,to,fto]
prob = ODEProblem(vinylCh, u0, vspan, p)
sol = solve(prob,Rosenbrock23())

┌ Warning: dt <= dtmin. Aborting. There is either an error in your model specification or the true solution is unstable.
└ @ SciMLBase /home/billyboy/.julia/packages/SciMLBase/DXiE6/src/integrator_interface.jl:345


retcode: DtLessThanMin
Interpolation: specialized 2nd order "free" stiffness-aware interpolation
t: 25-element Vector{Float64}:
 0.0
 9.981859021384082e-14
 1.19782308256609e-13
 2.2480973345067073e-13
 2.6713379772836127e-13
 3.8060890953425515e-13
 4.467046046710835e-13
 5.6862135049712e-13
 6.569730695104686e-13
 7.887731735100503e-13
 9.205732775096321e-13
 1.0714810820530825e-12
 1.171346916297733e-12
 1.2328066135495461e-12
 1.2712798613172607e-12
 1.29544122931934e-12
 1.310325155722978e-12
 1.3194072149437193e-12
 1.3249936111817359e-12
 1.3284300352867093e-12
 1.330523236147298e-12
 1.3317979581390916e-12
 1.3325793465499413e-12
 1.3330574492968374e-12
 1.3333484544825997e-12
u: 25-element Vector{Vector{Float64}}:
 [970.24, 970.24, 2910.75, 0.0, 0.0, 0.0, 0.0, 5.82, 500.0, 400.0, 2.0e6]
 [1251.7483067470303, 970.2399999999928, 2910.833089618299, 2.1569398413785366e-12, -0.08309140909783956, 4.9975177132128546e-12, 1.4882074678642822e-9, 5.778455190840959, 500.16994182862845, 3

In [69]:
k4b(300)

1.0049549714666183e14